# Python Object Oriented programming

## Object
- Everything is object in Python, and multiple names can be bound to the same object.
- What is the advantage of aliasing (bounding)? For mutable objects, like lists, passing their names to a function is nothing but pointing their objects as such if the function modifies the objects, the changes can be remain applied to the objects. It is something like pointers in other languages.

## Namespace
- Namespace is a mapping from names to objects.
    - All the built-ins in pythons. import built-ins; builtins.__dict__; it returns the namespace of the python built-ins
    - The global names in a module; modulename.__dict__ returns the namespace of the module.
    - Local names of a function. functionname.__dict__ returns the namespace of the function
    - In a nutshell, the set of attributes of an object forms a namespace. Attributes of an object includes both the names of variables and functions.
    - No relation between the names in different namespaces.
## Scope
- A scope is a textual region of a Python program where a namespace is directly accessible.
## Class
- As everything is object in Python, classes are also objects.
- When a class definition is entered, a new namespace is created.
- When a class definition is finished, a class object is creating, which is basically a wrapper around the namespace.
- The class object is bound to the class name defined in class definition header.
    -  The attributes (all the names of the namespace of the class object) can be referenced using class name following the classname.attribute, where attribute can be variable or function
    - Class can be instantiated using the callable. [Ex: Classname()]

## Instance Objects
- These are the instances of the classes
    - They can access two kids of attributes:
        - data variable attributes
        - method attributes

## Class variables Vs instance variables
- All the variables defined in a class are not instance variables.
- Instance variables can have scope of the instance
    - data unique to each instance
    - If an object cannot find a variable then it will look for class scope
    - However, the class cannot have access to the instance variables

- Class variables can be accessed using class or object names, and they can be shared to all the instances of the class

## Methods
### Instance methods
- They can alter the status (behaviour) of the instances
    - i.e. by having access to the instance variables as well as to the class variables
### Class methods
- They cannot alter the behaviour of an instance
    - i.e. they could not change the values of the instance variables. This is because the scope of the instance variables is limited to instances only but not to the class level.
        - Trying to access any instance variable inside a class method raises an error
- They can change the status of the class variables as such the status of the class.
- The can also use to create the new instances of the class by indirectly calling the __init__() constructor of a class. 
    - This can used as factor methods to create alternative way of the class instances.

### Static methods
- These methods cannot change neither the status of the class or object
    - i.e. these methods cannot have access to neither class variables or instance variables
- They can be used as utility methods, which can be used by either class methods or instance methods.
    - these methods can be called as, self.staticMethodName(), in instance methods because the instance methods have access to the namespace of the class, where the static methods are also the attributes of the class.

#### Example: 
##### Create a class for bank account which can support the functionalities such as
- accepts only unique bank account number
- first name and last name (using property)
- balance
- several types of transactions including
    - depositing money
    - withdrawing money
    - rejecting invalid transaction
    - paying interest
- Common interest for all the bank accounts
- generating confirmation code for all kinds of transactions
    - Transaction_type - Account_number - timestamp - transaction_number
- Include the steps to validate the input arguments

##### Create a class for a time zone object, which can be used to create timezone specific timestamp apart from the utc timestamp

In [2]:
import numbers
from datetime import datetime
from itertools import count
from datetime import datetime, timedelta
from collections import namedtuple

class BankAccount:
    acc_num_list = []
    interest_rate = 0
    transaction_id = {"Deposit": "D", 
                      "Withdraw": "W",
                      "Invalid": "X",
                      "Interest": "I",}
    
    transaction_num = count(100)

    def __init__(self, number, first_name='', last_name='', acc_balance=0, time_zone=None):
        if number in BankAccount.acc_num_list:
            raise ValueError("Bank Account Number must be unique")
        if not isinstance(number, numbers.Integral):
            raise ValueError("Bank Account Number must be integer")
        
        self._number = number
        self._first_name = first_name
        self._last_name = last_name
        self._balance = float(acc_balance)

        if time_zone==None:
            time_zone = TimeZone("UTC", 0, 0)
        self._time = datetime.utcnow()
        self._preferred_time = self._time + time_zone.offset

        BankAccount.acc_num_list.append(number)
        

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, first_name):
        self._first_name = first_name

    @property
    def lastname(self):
        return self._last_name

    @lastname.setter
    def lastname(self, last_name):
        self._last_name = last_name

    @property
    def name(self):
        return self._first_name+' '+self._last_name

    @property
    def balance(self):
        return self._balance

    @classmethod
    def interestrate(cls):
        return cls.interest_rate

    @classmethod
    def interestrate(cls, interest_rate):
        cls.interest_rate = interest_rate

    @staticmethod
    def generate_confirmation_code(transaction_id, acc_number, time, transaction_num): 
        items = [str(item) for item in [transaction_id, acc_number, time, transaction_num]]
        for item in items:
            BankAccount.str_len_None_validation(item)
        code = "-".join(items)
        return  code

    @staticmethod
    def  str_len_None_validation(item):
        if (len(item) == 0) or item is None:
            raise ValueError(f"{item} must not be empty or None!")

    @staticmethod
    def parse_confirmation_code(code, preferred_timeZone=None):
        items = code.split("-")
        if len(items) != 4:
            raise ValueError("The confirmation must have 4 fields!")
        
        transaction_code, account_number, raw_time, transcation_id = items
        time_utc = datetime.strptime(raw_time, "%Y%m%d%H%M%S")
        
        if preferred_timeZone is None:
            preferred_timeZone = TimeZone("UTC", 0, 0)
        time_preferred = time_utc + preferred_timeZone.offset
        
        return ConfirmationParse(transaction_code, account_number, time_utc, time_preferred, transcation_id)

    def deposit_money(self, amount):
        if not isinstance(self, BankAccount):
            raise ValueError("Object must be an instance of BankAccount")
        if (not isinstance(amount, numbers.Real)) or (amount<=0):
            raise ValueError("amount must be a double value and greater than 0!")

        code = self.generate_confirmation_code(self.transaction_id["Deposit"],
                                        self._number,
                                        datetime.utcnow().strftime("%Y%m%d%H%M%S"),
                                        next(self.transaction_num))
        
        self._balance += amount
        return code

    def withdraw_money(self, amount):
        if not isinstance(self, BankAccount):
            raise ValueError("Object must be an instance of BankAccount")
        if (not isinstance(amount, numbers.Real)) or (amount<=0):
            raise ValueError("amount must be a double value and greater than 0!")

        isValid = True
        if (self._balance - amount) > 0:
            tid = self.transaction_id["Withdraw"]
        else:
            tid = self.transaction_id["Invalid"]
            isValid = False

        code = self.generate_confirmation_code(tid,
                                        self._number,
                                        datetime.utcnow().strftime("%Y%m%d%H%M%S"),
                                        next(self.transaction_num))
        if isValid:
            self._balance -= amount
        return code


    def pay_interest(self):
        if not isinstance(self, BankAccount):
            raise ValueError("Object must be an instance of BankAccount")

        code = self.generate_confirmation_code(self.transaction_id["Interest"],
                                        self._number,
                                        datetime.utcnow().strftime("%Y%m%d%H%M%S"),
                                        next(self.transaction_num))

        if self._balance>0:
            self._balance = self._balance (1 + (self.interest_rate/100.0))
        return code

class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        
        # validate all the inputs
        if (name==None) or (len(str(name).strip())==0):
            raise ValueError("The name must not be None and empty!")
        
        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("The offset_hours must be integer type")

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("The offset_minutes must be integer type")

        if offset_hours>12 or offset_hours<-14:
            raise ValueError("The offset_hours must be in between -14 and 12 hours")

        if offset_minutes>59 or offset_hours<-59:
            raise ValueError("The offset_hours must be in between -59 and 59 minutes")

        self._name = name
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes

        offset = timedelta(hours=offset_hours, minutes=offset_minutes)

        self._offset = offset

    def __eq__(self, other):
        return (self._name==other._name and 
                  self._offset_hours==other._offset_hours and
                  self._offset_minutes==other._offset_minutes and
                  self._offset==other._offset)

    def __repr__(self):
        return (f"TimeZone(name='{self._name}', "
                f"offset_hours='{self._offset_hours}', "
                f"TimeZone(name='{self._offset_minutes}'")

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

ConfirmationParse = namedtuple("ConfirmationParse", ["transaction_code", "account_number", "time_utc", "time_preferred", "transcation_id"])

In [71]:
# Basic tests
acc1 = BankAccount(15, "Rathaiah", "Pureti")
acc1.deposit_money(100)
acc1.balance
acc1.withdraw_money(500)
BankAccount.interest_rate = 10

print(acc1.interest_rate)

The balance only 100 but you tried to withdraw more than that
10
